## Collaborative Filtering in 9 Lines of Code

http://tungwaiyip.info/2012/Collaborative%20Filtering.html

In [3]:
import numpy as np; 
import pandas as pd; 
from pandas import Series, DataFrame

rating = pd.read_csv('movie_rating.csv')
# rating.head()
rp = rating.pivot_table(columns=['critic'],index=['title'],values='rating')
rp

critic,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
title,,,,,,
Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN
Lady in the Water,NaN,3.0,3.0,2.5,3.0,NaN
Snakes on a Plane,3.5,3.5,4.0,3.5,4.0,4.5
Superman Returns,4.0,5.0,5.0,3.5,3.0,4.0
The Night Listener,4.5,3.0,3.0,3.0,3.0,NaN
You Me and Dupree,2.5,3.5,3.5,2.5,2.0,1.0


## Pearson correlation score



In [5]:
rating_toby = rp['Toby']
sim_toby = rp.corrwith(rating_toby)
sim_toby.sort_values()

critic
Gene Seymour     0.381246
Jack Matthews    0.662849
Claudia Puig     0.893405
Mick LaSalle     0.924473
Lisa Rose        0.991241
Toby             1.000000
dtype: float64

In [37]:
rating_c = rating.loc[rating_toby[rating.title].isnull().values & (rating.critic != 'Toby')]

rating_c.loc[:,'similarity'] = rating_c['critic'].map(sim_toby.get)
# rating_c['similarity'] = rating_c['critic'].map(sim_toby.get)

rating_c.loc[:,'sim_rating'] = rating_c.similarity * rating_c.rating

rating_c

C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,critic,title,rating,similarity,sim_rating
0,Jack Matthews,Lady in the Water,3.0,0.662849,1.988547
4,Jack Matthews,The Night Listener,3.0,0.662849,1.988547
5,Mick LaSalle,Lady in the Water,3.0,0.924473,2.773420
7,Mick LaSalle,Just My Luck,2.0,0.924473,1.848947
10,Mick LaSalle,The Night Listener,3.0,0.924473,2.773420
12,Claudia Puig,Just My Luck,3.0,0.893405,2.680215
15,Claudia Puig,The Night Listener,4.5,0.893405,4.020323
16,Lisa Rose,Lady in the Water,2.5,0.991241,2.478102
18,Lisa Rose,Just My Luck,3.0,0.991241,2.973722
20,Lisa Rose,The Night Listener,3.0,0.991241,2.973722


In [38]:
recommendation = rating_c.groupby('title').apply(lambda s: s.sim_rating.sum() / s.similarity.sum())
recommendation.sort_values(ascending=False)

title
The Night Listener    3.347790
Lady in the Water     2.832550
Just My Luck          2.530981
dtype: float64